In [ ]:
from google.colab import files
uploaded = files.upload()

In [33]:
import duckdb
# Assessment Question 1 - Of the industries where salaries and wages data did NOT exist in 2016 and only appeared later,
# which industry had the highest average value for actual filled jobs across time and what was that value.

result = duckdb.execute("""
with post_2016 as
(
    select
        distinct Series_title_2 as industry
    from 'financial_data.csv'
    where
        lower(Series_title_1) = 'salaries and wages'
        and cast(substr(cast(Period as string),1,4) as integer) >2016
        and Series_title_2 not in
                            (
                            select
                              Series_title_2
                            from 'financial_data.csv'
                            where
                                lower(Series_title_1) = 'salaries and wages'
                                and cast(substr(cast(Period as string),1,4) as integer) = 2016
                            )
)
,industry_filled_jobs as
(
  select
      Series_title_2 AS industry,
      AVG(Data_value) AS avg_filled_jobs
    from 'employment_data.csv'
    where
        "Group" = 'Industry by employment variable'
        and Series_title_1 = 'Filled jobs'
        and Series_title_3 = 'Actual'
        and Series_title_2 IN (select industry from post_2016)
    group by Series_title_2
)

select  * from industry_filled_jobs order  by avg_filled_jobs desc limit 1""")
df = result.fetchdf()
print(df)






       industry  avg_filled_jobs
0  Retail Trade    194053.711538


Saving employment_data.csv to employment_data.csv
